In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.decomposition import PCA,FastICA,FactorAnalysis,SparsePCA

import lightgbm as lgb
import xgboost as xgb

## 参数搜索和评价的
from sklearn.model_selection import GridSearchCV,cross_val_score,StratifiedKFold,train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from xgboost.sklearn import XGBRegressor
from lightgbm.sklearn import LGBMRegressor

In [36]:
train_set = pd.read_csv("../dataset/used_car_train_20200313.csv",sep=" ")
test_set = pd.read_csv("../dataset/used_car_testA_20200313.csv",sep=" ")

In [37]:
print(train_set.columns)
print(test_set.columns)

Index(['SaleID', 'name', 'regDate', 'model', 'brand', 'bodyType', 'fuelType',
       'gearbox', 'power', 'kilometer', 'notRepairedDamage', 'regionCode',
       'seller', 'offerType', 'creatDate', 'price', 'v_0', 'v_1', 'v_2', 'v_3',
       'v_4', 'v_5', 'v_6', 'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12',
       'v_13', 'v_14'],
      dtype='object')
Index(['SaleID', 'name', 'regDate', 'model', 'brand', 'bodyType', 'fuelType',
       'gearbox', 'power', 'kilometer', 'notRepairedDamage', 'regionCode',
       'seller', 'offerType', 'creatDate', 'v_0', 'v_1', 'v_2', 'v_3', 'v_4',
       'v_5', 'v_6', 'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12', 'v_13',
       'v_14'],
      dtype='object')


In [38]:
train_set = train_set.drop(["seller","offerType","name"],axis=1)
test_set = test_set.drop(["seller","offerType","name"],axis=1)

In [39]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 28 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   SaleID             150000 non-null  int64  
 1   regDate            150000 non-null  int64  
 2   model              149999 non-null  float64
 3   brand              150000 non-null  int64  
 4   bodyType           145494 non-null  float64
 5   fuelType           141320 non-null  float64
 6   gearbox            144019 non-null  float64
 7   power              150000 non-null  int64  
 8   kilometer          150000 non-null  float64
 9   notRepairedDamage  150000 non-null  object 
 10  regionCode         150000 non-null  int64  
 11  creatDate          150000 non-null  int64  
 12  price              150000 non-null  int64  
 13  v_0                150000 non-null  float64
 14  v_1                150000 non-null  float64
 15  v_2                150000 non-null  float64
 16  v_

In [40]:
for i in train_set.columns:
    print(i,train_set[i].unique().size)

SaleID 150000
regDate 3894
model 249
brand 40
bodyType 9
fuelType 8
gearbox 3
power 566
kilometer 13
notRepairedDamage 3
regionCode 7905
creatDate 96
price 3763
v_0 143997
v_1 143998
v_2 143997
v_3 143998
v_4 143998
v_5 139624
v_6 109766
v_7 138709
v_8 142451
v_9 140617
v_10 143997
v_11 143997
v_12 143997
v_13 143998
v_14 143998


In [41]:
#power
def outliers_proc(data, col_name, scale=3):
    """
        用于截尾异常值， 默认用box_plot(scale=3)进行清洗
        param:
            data：接收pandas数据格式
            col_name: pandas列名
            scale: 尺度
    """
    data_col = data[col_name]
    Q1 = data_col.quantile(0.25) # 0.25分位数
    Q3 = data_col.quantile(0.75)  # 0,75分位数
    IQR = Q3 - Q1

    data_col[data_col < Q1 - (scale * IQR)] = Q1 - (scale * IQR)
    data_col[data_col > Q3 + (scale * IQR)] = Q3 + (scale * IQR)

    return data[col_name]

train_set["power"] = outliers_proc(train_set,'power')

D:\anaconda\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
D:\anaconda\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [49]:
'''
bodyType             4506
fuelType             8680
gearbox              5981
'''
print(train_set.gearbox.mode())
train_set["bodyType"] = train_set.bodyType.fillna(-1)
train_set["fuelType"] = train_set.fuelType.fillna(train_set.fuelType.mode())
train_set["gearbox"] = train_set.gearbox.fillna(train_set.gearbox.mode())

test_set["bodyType"] = test_set.bodyType.fillna(test_set.bodyType.mode())
test_set["fuelType"] = test_set.fuelType.fillna(test_set.fuelType.mode())
test_set["gearbox"] = test_set.gearbox.fillna(test_set.gearbox.mode())

0    0.0
dtype: float64


In [51]:
train_set["gearbox"].value_counts()

0.0    111623
1.0     32396
Name: gearbox, dtype: int64

In [9]:
train_set["train"] = 1
test_set["train"] = 0
data = pd.concat([train_set,test_set],ignore_index=True)

In [10]:
data

,SaleID,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,notRepairedDamage,...,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14,train
0,0,20040402,30.0,6,1.0,0.0,0.0,60,12.5,0.0,...,0.101988,0.129549,0.022816,0.097462,-2.881803,2.804097,-2.420821,0.795292,0.914762,1
1,1,20030301,40.0,1,2.0,0.0,0.0,0,15.0,-,...,0.121004,0.135731,0.026597,0.020582,-4.900482,2.096338,-1.030483,-1.722674,0.245522,1
2,2,20040403,115.0,15,1.0,0.0,0.0,163,12.5,0.0,...,0.114912,0.165147,0.062173,0.027075,-4.846749,1.803559,1.565330,-0.832687,-0.229963,1
3,3,19960908,109.0,10,0.0,0.0,1.0,193,15.0,0.0,...,0.110300,0.121964,0.033395,0.000000,-4.509599,1.285940,-0.501868,-2.438353,-0.478699,1
4,4,20120103,110.0,5,1.0,0.0,0.0,68,5.0,0.0,...,0.073205,0.091880,0.078819,0.121534,-1.896240,0.910783,0.931110,2.834518,1.923482,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,199995,19960503,4.0,4,4.0,0.0,0.0,116,15.0,0.0,...,0.130044,0.049833,0.028807,0.004616,-5.978511,1.303174,-1.207191,-1.981240,-0.357695,0
199996,199996,19991011,0.0,0,0.0,0.0,0.0,75,15.0,0.0,...,0.108095,0.066039,0.025468,0.025971,-3.913825,1.759524,-2.075658,-1.154847,0.169073,0
199997,199997,20040412,49.0,1,0.0,1.0,1.0,224,15.0,0.0,...,0.105724,0.117652,0.057479,0.015669,-4.639065,0.654713,1.137756,-1.390531,0.254420,0
199998,199998,20020008,27.0,1,0.0,0.0,1.0,334,15.0,0.0,...,0.000490,0.137366,0.086216,0.051383,1.833504,-2.828687,2.465630,-0.911682,-2.057353,0


In [11]:
data['used_time'] = (pd.to_datetime(data['creatDate'], format='%Y%m%d', errors='coerce') - 
                            pd.to_datetime(data['regDate'], format='%Y%m%d', errors='coerce')).dt.days

In [12]:
# data['used_time'].isnull().sum()

In [13]:
# data = data.drop(["SaleID","regDate","creatDate"],axis=1)
data = data.drop(["SaleID"],axis=1)

In [14]:
data["notRepairedDamage"].replace("-",np.nan,inplace=True)

In [15]:
data["notRepairedDamage"] = data["notRepairedDamage"].astype("float")

In [16]:
data["notRepairedDamage"] = data["notRepairedDamage"].fillna(-1)

In [17]:
data["damage"] = data["used_time"]+data["kilometer"]

In [18]:
train_data = data[data["train"]==1]
test_data = data[data["train"]==0]


In [19]:
train_data = train_data.drop(["train","price"],axis=1)
test_data = test_data.drop(["train","price"],axis=1)

In [20]:
X_data = pd.get_dummies(train_data)
X_test = pd.get_dummies(test_data)

In [52]:
X_data = X_data.fillna(-1)
X_test = X_test.fillna(-1)

In [53]:
y = train_set["price"]

In [54]:
train_x = X_data[:int(len(X_data)*0.8)]
valid_x = X_data[int(len(X_data)*0.8):]
train_y = y[:int(len(X_data)*0.8)]
valid_y = y[int(len(X_data)*0.8):]

In [55]:
x_train,x_test,y_train,y_test = train_test_split(train_x,train_y,test_size = 0.2)

In [56]:
x_train.isnull().sum()

regDate              0
model                0
brand                0
bodyType             0
fuelType             0
gearbox              0
power                0
kilometer            0
notRepairedDamage    0
regionCode           0
creatDate            0
v_0                  0
v_1                  0
v_2                  0
v_3                  0
v_4                  0
v_5                  0
v_6                  0
v_7                  0
v_8                  0
v_9                  0
v_10                 0
v_11                 0
v_12                 0
v_13                 0
v_14                 0
used_time            0
damage               0
dtype: int64

In [57]:
max_depth=15

In [58]:
model = SVR()
model.fit(x_train,y_train)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [60]:
mean_absolute_error(model.predict(x_test),y_test)

4503.94640035785

In [105]:
X_data.columns == X_test.columns


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [106]:
## xgb-Model

xgr = XGBRegressor(n_estimators=120, learning_rate=0.09, gamma=0, subsample=1,\
        colsample_bylevel=0.4, max_depth=15,objective ='reg:squarederror') #,objective ='reg:squarederror'

scores_train = []
scores = []

## 5折交叉验证方式
sk=StratifiedKFold(n_splits=10,shuffle=True,random_state=0)
for train_ind,val_ind in sk.split(X_data,y):
    
    train_x=X_data.iloc[train_ind].values
    train_y=y.iloc[train_ind]
    val_x=X_data.iloc[val_ind].values
    val_y=y.iloc[val_ind]
    
    xgr.fit(train_x,train_y)
    pred_train_xgb=xgr.predict(train_x)
    pred_xgb=xgr.predict(val_x)
    
    score_train = mean_absolute_error(train_y,pred_train_xgb)
    scores_train.append(score_train)
    score = mean_absolute_error(val_y,pred_xgb)
    scores.append(score)

print('Train mae:',np.mean(score_train))
print('Val mae',np.mean(scores))

D:\anaconda\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


Train mae: 107.20599530483175
Val mae 530.675681982229


In [107]:
def build_model_xgb(x_train,y_train):
    model = XGBRegressor(n_estimators=150, learning_rate=0.09, gamma=0, subsample=1,\
        colsample_bylevel=0.4, max_depth=15,objective ='reg:squarederror') #, objective ='reg:squarederror'
    model.fit(x_train, y_train)
    return model

def build_model_lgb(x_train,y_train):
    estimator = lgb.LGBMRegressor(num_leaves=127,n_estimators = 150)
    param_grid = {
        'learning_rate': [0.01, 0.05, 0.1, 0.2],
    }
    gbm = GridSearchCV(estimator, param_grid)
    gbm.fit(x_train, y_train)
    return gbm

In [108]:
x_train,x_val,y_train,y_val = train_test_split(X_data,y,test_size=0.3)

In [109]:
# print('Train lgb...')
# model_lgb = build_model_lgb(x_train,y_train)
# val_lgb = model_lgb.predict(x_val)
# MAE_lgb = mean_absolute_error(y_val,val_lgb)
# print('MAE of val with lgb:',MAE_lgb)

# print('Predict lgb...')
# model_lgb_pre = build_model_lgb(X_data,y)
# subA_lgb = model_lgb_pre.predict(X_test)

In [110]:
# print('Train xgb...')
# model_xgb = build_model_xgb(x_train,y_train)
# val_xgb = model_xgb.predict(x_val)
# MAE_xgb = mean_absolute_error(y_val,val_xgb)
# print('MAE of val with xgb:',MAE_xgb)

# print('Predict xgb...')
# model_xgb_pre = build_model_xgb(X_data,y)
# subA_xgb = model_xgb_pre.predict(X_test)


In [111]:
# val_Weighted = (1-MAE_lgb/(MAE_xgb+MAE_lgb))*val_lgb+(1-MAE_xgb/(MAE_xgb+MAE_lgb))*val_xgb
# val_Weighted[val_Weighted<0]=10 # 由于我们发现预测的最小值有负数，而真实情况下，price为负是不存在的，由此我们进行对应的后修正
# print('MAE of val with Weighted ensemble:',mean_absolute_error(y_val,val_Weighted))